In [1]:
import os

In [2]:
%pwd

'c:\\Users\\MONIKA\\Deep_Learning_Projects\\NLP_Projects\\2_Text_Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\MONIKA\\Deep_Learning_Projects\\NLP_Projects\\2_Text_Summarizer'

## entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

## configurations

In [7]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config


## componenets

In [11]:
!pip install evaluate

In [12]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
#from datasets import load_dataset, load_from_disk, load_metric
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

## evaluation

In [21]:
my_device = "cuda" if torch.cuda.is_available() else "cpu"
my_device

'cpu'

In [31]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        It yeild successive batch-sized chunks from list of elements.
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i+batch_size]


    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size = 16, 
                                    device = "cuda" if torch.cuda.is_available() else "cpu",
                                    column_text = "article",
                                    column_summary = "highlights"):

        article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), 
            total = len(article_batches)):

            inputs = tokenizer(article_batch, max_length = 1024, truncation = True,
                            padding = 'max_length', return_tensors = 'pt')

            summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                    attention_mask = inputs['attention_mask'].to(device),
                                    length_penalty = 0.8, num_beams = 8, max_length = 128)

            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the token, and add the decoded texts with the references to the metric.

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True,
                                                clean_up_tokenization_spaces = True)
                                                for s in summaries]

            metric.add_batch(predictions = decoded_summaries, references = target_batch)

            # Finally compute and return the ROUGE score
            score = metric.compute()

        return score



    def evaluate(self):
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum['test'][0:10],
            rouge_metric, 
            trainer.model,
            tokenizer, 
            batch_size = 2,
            column_text = 'dialogue',
            column_summary = 'summary'
        )

        rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index = [f'pegasus'])
        df.to_csv(self.config.metric_file_name, index = False)

## pipeline

In [32]:
try:
    configM = ConfigurationManager()
    model_evaluation_config = configM.get_model_evaluation_config()
    model_evaluation_obj = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_obj.evaluate()

except Exception as e:
    raise e

[2025-10-29 15:18:04,388: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-29 15:18:04,392: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-29 15:18:04,393: INFO: common: Created directory at: artifacts]
[2025-10-29 15:18:04,396: INFO: common: Created directory at: artifacts/model_evaluation]


ValueError: Unrecognized model in artifacts/model_trainer. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, blt, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_anything, depth_pro, d